In [1]:
from helpers.training import TrainingData

training_data = TrainingData()

/home/justin/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [2]:
# currently, running PCA makes it worse - need to investigate why
# from sklearn.decomposition import PCA

# # figure out n_components later
# pca = PCA()

# X_tr = pca.fit_transform(training_data.X_tr)
# X_ts = pca.fit_transform(training_data.X_ts)

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gbt = GradientBoostingClassifier(max_features="log2")
gbt.fit(training_data.X_tr, training_data.Y_tr)

p = gbt.predict(training_data.X_ts)

precision = accuracy_score(p, training_data.Y_ts) * 100
print("Accuracy using GB: {0:.2f}%".format(precision))

Accuracy using GB: 60.72%


In [20]:
from sklearn.ensemble import RandomForestClassifier

rft = RandomForestClassifier()
rft.fit(X_tr, training_data.Y_tr)

p = rft.predict(X_ts)

precision = accuracy_score(p, training_data.Y_ts) * 100
print("Accuracy using RF: {0:.2f}%".format(precision))

Accuracy using RF: 44.91%


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

jobs=-1

def svc_param_selection(X, y, n_folds, n_jobs):
    params = {'C': stats.uniform(0, 10),
          'gamma': stats.uniform(0, 1)}
    rand_search = RandomizedSearchCV(SVC(),
                                     param_distributions=params,
                                     cv=n_folds,
                                     n_jobs=jobs,
                                     random_state=2017)
    rand_search.fit(X, y)
    print(rand_search.best_params_)
    return rand_search.best_params_

best_params = svc_param_selection(training_data.features, training_data.Y, 3, 4)

svc = SVC(C=best_params['C'], gamma=best_params['gamma'])
svc.fit(X_tr, training_data.Y_tr)

p = svc.predict(X_ts)

precision = accuracy_score(p, training_data.Y_ts) * 100
print("Accuracy using SVC: {0:.2f}%".format(precision))